<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/team_7_lab04/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (94.4 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of

In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, to_date, year, expr

In [73]:
# Crear la sesión de Spark
spark = SparkSession.builder.appName("Movies-Activity").getOrCreate()

In [74]:
# Leer el archivo JSON
df_movies = spark.read.option("inferSchema", "true").json("movies.json")

In [75]:
# 1. Calcular margen de beneficio (Profit Margin)
df_movies = df_movies.withColumn("Profit_Margin",(col("Worldwide_Gross") - col("Production_Budget")) / col("Production_Budget"))

df_movies.select("Title", "Profit_Margin").show()

+--------------------+-------------------+
|               Title|      Profit_Margin|
+--------------------+-------------------+
|      The Land Girls|       -0.981739625|
|First Love, Last ...|-0.9637466666666666|
|I Married a Stran...|          -0.187464|
|Let's Talk About Sex|0.24538333333333334|
|                Slam|           0.087521|
| Mississippi Mermaid|        0.640344375|
|           Following|  6.450833333333334|
|             Foolish|          2.7668175|
|             Pirates|       -0.841454375|
|     Duel in the Sun|                2.4|
|           Tom Jones|               36.6|
|             Oliver!|          2.7402877|
|To Kill A Mocking...|           5.564923|
|    Tora, Tora, Tora|         0.18193164|
|   Hollywood Shuffle|           51.28617|
|Over the Hill to ...|               29.0|
|              Wilson|-0.6153846153846154|
|        Darling Lili|-0.7727272727272727|
|The Ten Commandments|  4.925925925925926|
|        12 Angry Men|               -1.0|
+----------

In [76]:
# 2. Determinar si es un éxito de taquilla (Box Office Hit)
df_movies = df_movies.withColumn("Box_Office_Hit",when(col("Worldwide_Gross") > 2 * col("Production_Budget"), "Hit").otherwise("Flop"))

df_movies.select("Title", "Box_Office_Hit").show()

+--------------------+--------------+
|               Title|Box_Office_Hit|
+--------------------+--------------+
|      The Land Girls|          Flop|
|First Love, Last ...|          Flop|
|I Married a Stran...|          Flop|
|Let's Talk About Sex|          Flop|
|                Slam|          Flop|
| Mississippi Mermaid|          Flop|
|           Following|           Hit|
|             Foolish|           Hit|
|             Pirates|          Flop|
|     Duel in the Sun|           Hit|
|           Tom Jones|           Hit|
|             Oliver!|           Hit|
|To Kill A Mocking...|           Hit|
|    Tora, Tora, Tora|          Flop|
|   Hollywood Shuffle|           Hit|
|Over the Hill to ...|           Hit|
|              Wilson|          Flop|
|        Darling Lili|          Flop|
|The Ten Commandments|           Hit|
|        12 Angry Men|          Flop|
+--------------------+--------------+
only showing top 20 rows



In [77]:
# 3. Convertir Release Date a un formato estándar y extraer el año
df_movies = df_movies.withColumn(
    "Release_Date_Formatted",
    when(col("Release_Date").rlike("^\d{2}-[A-Za-z]{3}-\d{2}$"), to_date(col("Release_Date"), "dd-MMM-yy"))
    .when(col("Release_Date").rlike("^\d{4}-\d{2}-\d{2}$"), to_date(col("Release_Date"), "yyyy-MM-dd"))
)

df_movies = df_movies.withColumn(
    "Release_Date",
    when(year(col("Release_Date_Formatted")) > 2024, expr("add_months(Release_Date_Formatted, -1200)"))  # Restar 100 años (1200 meses)
    .otherwise(col("Release_Date_Formatted"))
)

df_movies = df_movies.withColumn("Release_Year", year(col("Release_Date")))

df_movies.select("Title", "Release_Date", "Release_Year").show()

+--------------------+------------+------------+
|               Title|Release_Date|Release_Year|
+--------------------+------------+------------+
|      The Land Girls|  1998-06-12|        1998|
|First Love, Last ...|        NULL|        NULL|
|I Married a Stran...|  1998-08-28|        1998|
|Let's Talk About Sex|  1998-09-11|        1998|
|                Slam|        NULL|        NULL|
| Mississippi Mermaid|  1999-01-15|        1999|
|           Following|        NULL|        NULL|
|             Foolish|        NULL|        NULL|
|             Pirates|        NULL|        NULL|
|     Duel in the Sun|  1946-12-31|        1946|
|           Tom Jones|        NULL|        NULL|
|             Oliver!|  1968-12-11|        1968|
|To Kill A Mocking...|  1962-12-25|        1962|
|    Tora, Tora, Tora|  1970-09-23|        1970|
|   Hollywood Shuffle|        NULL|        NULL|
|Over the Hill to ...|  2020-09-17|        2020|
|              Wilson|        NULL|        NULL|
|        Darling Lil

In [78]:
# 4. Categorizar calificaciones de IMDB (IMDB_Rating_Category)
df_movies = df_movies.withColumn(
    "IMDB_Rating_Category",
    when(col("IMDB_Rating") >= 7.0, "High")
    .when((col("IMDB_Rating") >= 5.0) & (col("IMDB_Rating") < 7.0), "Medium")
    .otherwise("Low")
)

df_movies.select("Title", "IMDB_Rating", "IMDB_Rating_Category").show()

+--------------------+-----------+--------------------+
|               Title|IMDB_Rating|IMDB_Rating_Category|
+--------------------+-----------+--------------------+
|      The Land Girls|        6.1|              Medium|
|First Love, Last ...|        6.9|              Medium|
|I Married a Stran...|        6.8|              Medium|
|Let's Talk About Sex|       NULL|                 Low|
|                Slam|        3.4|                 Low|
| Mississippi Mermaid|       NULL|                 Low|
|           Following|        7.7|                High|
|             Foolish|        3.8|                 Low|
|             Pirates|        5.8|              Medium|
|     Duel in the Sun|        7.0|                High|
|           Tom Jones|        7.0|                High|
|             Oliver!|        7.5|                High|
|To Kill A Mocking...|        8.4|                High|
|    Tora, Tora, Tora|       NULL|                 Low|
|   Hollywood Shuffle|        6.8|              

In [79]:
# 5. Calcular el promedio de calificación de IMDB por distribuidor
df_movies = df_movies.withColumn(
    "IMDB_Rating_Category",
    when(col("IMDB_Rating") >= 7.0, "High")
    .when((col("IMDB_Rating") >= 5.0) & (col("IMDB_Rating") < 7.0), "Medium")
    .otherwise("Low")
)

df_movies.select("Title", "IMDB_Rating", "IMDB_Rating_Category").show()

+--------------------+-----------+--------------------+
|               Title|IMDB_Rating|IMDB_Rating_Category|
+--------------------+-----------+--------------------+
|      The Land Girls|        6.1|              Medium|
|First Love, Last ...|        6.9|              Medium|
|I Married a Stran...|        6.8|              Medium|
|Let's Talk About Sex|       NULL|                 Low|
|                Slam|        3.4|                 Low|
| Mississippi Mermaid|       NULL|                 Low|
|           Following|        7.7|                High|
|             Foolish|        3.8|                 Low|
|             Pirates|        5.8|              Medium|
|     Duel in the Sun|        7.0|                High|
|           Tom Jones|        7.0|                High|
|             Oliver!|        7.5|                High|
|To Kill A Mocking...|        8.4|                High|
|    Tora, Tora, Tora|       NULL|                 Low|
|   Hollywood Shuffle|        6.8|              

In [80]:
spark.stop()